In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os
import stat
import glob
from time import sleep
from pathlib import Path

dispatch_dir = '/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/scripts/eval_20220712/dispatcher.py'
save_dir = '/n/data1/hms/neurobio/sabatini/josh/analysis/gce2'
eval_param_py = '/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/scripts/eval_20220712/eval-params-sweep.py'
mouse_stat_dir = '/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/data/training/mouse2_6__20210409'
# param_sweep_dir = '/n/data1/hms/neurobio/sabatini/rich/analysis/ROI_net_training/20220511_SimCLR_penaltyOrth'

python_params = []
# sweep_folders = glob.glob('/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/20220630*')
sweep_folders = glob.glob('/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/20220713*')

for param_sweep_dir in sweep_folders:
    slurm_name = param_sweep_dir.split('/')[-1]
#     eval_dir = str(Path(save_dir) / slurm_name)
#     print(eval_dir)
#     print("....>", dispatch_dir, save_dir, eval_param_py, eval_dir, "job_name", mouse_stat_dir, param_sweep_dir)
#     run dispatch_dir, save_dir, eval_param_py, eval_dir, "job_name", mouse_stat_dir, param_sweep_dir
#     !python3 $dispatch_dir $save_dir $eval_param_py $eval_dir "job_name" $mouse_stat_dir $param_sweep_dir
#     os.system(exec_string)
#     print(exec_string)
#     print()
    python_params.append(f"python3 {dispatch_dir} {save_dir} {eval_param_py} {slurm_name} job_name {mouse_stat_dir} {param_sweep_dir}")


log_fn = 'eval.log'

pp_str = '\n'.join(python_params)
exec_file = \
f"""#!/usr/bin/bash
#SBATCH --job-name=eval_start
#SBATCH --gres=gpu:rtx8000:1
#SBATCH --partition=gpu_quad
#SBATCH -c 1
#SBATCH -n 1
#SBATCH --mem=100GB
#SBATCH --time=0-00:45:00
unset XDG_RUNTIME_DIR
cd /n/data1/hms/neurobio/sabatini/josh/analysis/gce/
date
echo "loading modules"
source /etc/profile.d/modules.sh
echo "activating environment"
source activate ROI_env
echo "starting job"
{pp_str}
"""
#module load gcc/9.2.0

# print(exec_file)

# exec_fn = str(Path(save_dir) / 'eval.sh')
# with open(exec_fn, 'w') as sb:
#     sb.write(exec_file)
#     st = os.stat(exec_fn)
#     os.chmod(exec_fn, st.st_mode | stat.S_IEXEC)


# os.system(f'srun {exec_fn}')


exec_fn = str(Path(save_dir) / 'eval.sh')
with open(exec_fn, 'w') as sb:
    sb.write(exec_file)
    st = os.stat(exec_fn)
    os.chmod(exec_fn, st.st_mode | stat.S_IEXEC)

# os.system(f'srun -t 1 -p priority {exec_fn}')
os.system(f'sbatch {exec_fn}')

sbatch: Setting QOS to gpuquad_qos


0

Submitted batch job 57531395


In [3]:
import glob
sleep(1)
for log in sorted(glob.glob('/n/data1/hms/neurobio/sabatini/josh/analysis/gce/*/job_name*/*.log')):
    print('log:', log)
    print('----------')
    with open(log, 'r') as logf:
        log_str = logf.read()
    if 'Incorrect' in log_str:
        print(log_str)
    print('==========')


log: /n/data1/hms/neurobio/sabatini/josh/analysis/gce/20220630_SimCLR_best/job_name0/print_log_57086128.log
----------
Wed Jul 13 01:13:52 EDT 2022
loading modules
activating environment
/var/spool/slurmd/job57086128/slurm_script: line 16: activate: No such file or directory
starting job
1.21.0
0.12.0+cu113
script environment: jupyter_launcher
params_eval {'github_loc': '/n/data1/hms/neurobio/sabatini/josh/github_repos', 'path_stat': '/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/data/training/mouse2_6__20210409/stat.npy', 'input_dir': '/n/data1/hms/neurobio/sabatini/josh/analysis/ROI_net_training/20220630_SimCLR_best', 'path_labels': ['/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/data/training/mouse2_6__20210409/labels_round2_sesh1.npy', '/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/data/training/mouse2_6__20210409/labels_round2_sesh2.npy'], 'model_loc': '/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_RO

In [4]:
import glob
import shutil

sleep(1)
for report in sorted(glob.glob('/n/data1/hms/neurobio/sabatini/josh/analysis/gce/*/job_name*/*.png')):
    print('report:', report)
    
    output_setup = report.replace('/job_name0/', '/').replace('report_out_jobnum=', '').split('/gce/')
    output_name = output_setup[-1].replace('/', '-')
    output_file = str(Path(output_setup[0]) / 'gce' / 'final_reports' / output_name)
#     print(output_file)
    shutil.copyfile(report, output_file)

report: /n/data1/hms/neurobio/sabatini/josh/analysis/gce/20220630_SimCLR_temp/job_name0/report_out_jobnum=jobNum_0.png


FileNotFoundError: [Errno 2] No such file or directory: '/n/data1/hms/neurobio/sabatini/josh/analysis/gce/final_reports/20220630_SimCLR_temp-jobNum_0.png'

In [ ]:
# import json

# path_params = 

# with open(path_params, 'r') as f:
#     params_eval = json.load(f)

# input_dir = params_eval['input_dir']

In [ ]:
# sorted(glob.glob(str(Path(input_dir) / 'jobNum_*')))